# Pipeline on DAP for simulation results

#### Specifically for calcium pathway simulation

In [1]:
%set -J 1 -j 2
%run dap
[dap_1]
depends: Py_Module("pandasql"), Py_Module('fisher'), Py_Module('plotly')
parameter: sample_size = 2000
parameter: seed = [999]
parameter: prevalence = [0.01]
parameter: gamma = [(5,1)]
parameter: name = "data/calcium_pathway_sos"
input: for_each = ["gamma", "prevalence", "seed"]
output: "${name}_%s.data.pkl" % "N{}_p{}".format(int("${sample_size}")*2, "${_prevalence}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from Simulation_calcium import Environment, run_simulation
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, "N{}_p{}".format(${sample_size} * 2, ${_prevalence}))

[dap_2]
depends: Py_Module("feather-format")
input: group_by = 'single'
output: "${_input!n}.feather"
task:
python:
import sys, os
sys.path.append(os.getcwd())
import pickle, feather
import pandas as pd
import numpy as np
from Simulation_calcium import Environment, load_reference_gene 
args = Environment()
dat = pickle.load(open(${_input!r}, "rb"))
ref = load_reference_gene(args["refgene_file"])
# create regression data
genes = pd.Series(list(set(ref['gene_name'])))
regression_data = np.array([np.array(genes.isin(item["gene_name"]), dtype = float) for item in dat['case'] + dat['ctrl']])
regression_data = np.hstack((np.matrix([1] * len(dat['case']) + [0] * len(dat['ctrl'])).T, regression_data))
mask = ~np.ravel((regression_data==0).all(0))
regression_data = regression_data[:,mask]
# save to disk
feather.write_dataframe(pd.DataFrame(regression_data, columns = ['phenotype'] + genes[mask[1:]].tolist()), ${_output!r})

[dap_3]
depends: 
input: 
output: 
task:
python:


Set sos options to "-J 1 -j 2"


"<i id=""status_localhost_fcf69a1cd086731cad449eb56af1bfa1"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_fcf69a1cd086731cad449eb56af1bfa1').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_fcf69a1cd086731cad449eb56af1bfa1').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('fcf69a1cd086731cad449eb56af1bfa1', 'localhost')"" >",fcf69a1cd086731cad449eb56af1bfa1,,0 sec


In [ ]:
[varbvs_1]
depends: Py_Module("pandasql"), Py_Module('fisher'), Py_Module('plotly')
parameter: sample_size = 2000
parameter: seed = [12]
parameter: prevalence = [0.01]
parameter: gamma = [(3,1), (5,1), (9,1)]
parameter: n_causal_gene = [100, 200]
parameter: name = 'data/del_sim'
input: for_each = ['gamma', 'prevalence', 'n_causal_gene', 'seed']
output: "${name}_%s.data.pkl" % "shape{}_scale{}_p{}_N{}_M{}_{}".format("${_gamma[0]}", "${_gamma[1]}", "${_prevalence}", int("${sample_size}") * 2, "${_n_causal_gene}", "${_seed}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from simulation import Environment, run_simulation
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['n_causal_gene'] = ${_n_causal_gene}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, "shape{}_scale{}_p{}_N{}_M{}_{}".format(${_gamma[0]}, ${_gamma[1]}, ${_prevalence}, ${sample_size} * 2, ${_n_causal_gene}, ${_seed}))